In [1]:
from collections import Counter, defaultdict
import copy
from functools import partial
import itertools
import os
from pathlib import Path
import random
import re
import string
import time
from typing import Dict, List, Tuple
import warnings

from gensim.models import Word2Vec
import joblib
from numba import cuda
import numpy as np
import pandas as pd
import plotly_express as px
import scipy

from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GroupKFold, KFold
from sklearn.utils import shuffle

import tensorflow as tf
import tensorflow_hub as hub

import torch
from torch import nn
from torch.optim.optimizer import Optimizer
from torch.utils.data import Dataset, Sampler, DataLoader

先用fasttext.pkl预训练出来的词向量，然后w2vec进行训练，训练完之后再放入lstm之中

整个模型操作的神奇之处在于使用了fasttext+w2vec+lstm的相应的模型

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *
from keras.backend.tensorflow_backend import set_session
gpu_no = '0' # or '1'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))
#设置程序按需占用gpu显存

Using TensorFlow backend.


In [3]:
start_time = time.time()
SAMPLE_SUBMISSION_PATH = '/home/huarui/competitiondata/google-qa-labeling-lstm/sample_submission.csv'
#获取对应的sample_submission.csv
USE_DIR = '/home/huarui/competitiondata/google-qa-labeling-lstm/universal-sentence-encoder-qa/'
EMBEDDING_PATH = '/home/huarui/competitiondata/google-qa-labeling-lstm/fasttext.pkl'

TRAIN_PATH = '/home/huarui/competitiondata/google-qa-labeling-lstm/train.csv'
TEST_PATH = '/home/huarui/competitiondata/google-qa-labeling-lstm/test.csv'

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

num_targets = 30
target_names = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
    'answer_helpful',
    'answer_level_of_information',
    'answer_plausible',
    'answer_relevance',
    'answer_satisfaction',
    'answer_type_instructions',
    'answer_type_procedure',
    'answer_type_reason_explanation',
    'answer_well_written']

train_df['question'] = train_df['question_title'] + ' ' + train_df['question_body']
test_df['question'] = test_df['question_title'] + ' ' + test_df['question_body']
#注意是question中的title与body进行融合
n_splits = 5
n_epochs = 9
batch_size = 128

max_q_len = 512
max_a_len = 512

updates_per_epoch = 100
mu = 0.9

embed_size = 300
max_features = 60000

seed = 1029
device = torch.device('cuda')

ps = PorterStemmer()
#将不同形式的单词转换为同一形式
#比如create和created都得到creat
lc = LancasterStemmer()
sb = SnowballStemmer('english')
#snowballstemmer是一款非常瘦小的语言转换库，支持15种语言
#'english'为其中一种语言
#这里面都使用的nltk,nltk是python很强大的第三方库，可以很方便的
#完成很多NLP的任务，包括分词、词性标注、命名实体标注ner及句法分析

warnings.filterwarnings('ignore')
#过滤掉所有的警告信息
random.seed(seed)
#上面的seed = 1029，所以使用random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
#确定性能的变化是来自模型还是数据集的变化，或者仅仅是
#一些新的随机样本点带来的结果
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
#获取对应的fasttest.pkl的相应的文件内容

In [4]:
print(train_df.head())
print('$$$$$$$$$$$$$$$$$$$$$')
print(test_df.head())

   qa_id                                     question_title  \
0      0  What am I losing when using extension tubes in...   
1      1  What is the distinction between a city and a s...   
2      2  Maximum protusion length for through-hole comp...   
3      3              Can an affidavit be used in Beit Din?   
4      5       How do you make a binary image in Photoshop?   

                                       question_body question_user_name  \
0  After playing around with macro photography on...               ysap   
1  I am trying to understand what kinds of places...      russellpierce   
2  I'm working on a PCB that has through-hole com...          Joe Baker   
3  An affidavit, from what i understand, is basic...         Scimonster   
4  I am trying to make a binary image. I want mor...            leigero   

                                  question_user_page  \
0         https://photo.stackexchange.com/users/1024   
1           https://rpg.stackexchange.com/users/8774   
2 

In [5]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}


def replace_typical_misspell(text: str) -> str:
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    #re.compile函数用于编译正则表达式，生成一个pattern对象
    #中间的%s代表赋值用法,赋值的内容为'|'.join(misspell_dict.keys())
    #正则表达式中的'|'表示或，这里代表的含义为能够匹配第一个keys()
    #或第二个keys()或第三个keys()依次类推
    
    #''.join([a,b]),将[a,b]字符串进行连接
    def replace(match):
        return misspell_dict[match.group(0)]
    #match.group(0)就是匹配正则表达式的整体结果
    #misspell_dict[match.group(0)]是匹配正则式之后进行替换的结果字符串

    return misspell_re.sub(replace, text)
    #1.先去replace之中找寻对应的替换规则，对text字符串
    #使用replace函数进行替换
    #2.进入replace函数之中以后，再使用misspell_dict[match.group(0)]
    #这里面调用match.group(0)之时使用上面的misspell_re进行正则表达式定义
    #3.正则表达式匹配出来之后，使用misspell_dict找寻替换的对应内容
    #然后返回最外面的misspell_re.sub替换层，将text所有内容都进行替换
    
    #!!!这里的replace函数调用的就是外面的内容misspell_re对应的正则
    #替换的内容


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']


def clean_text(text: str) -> str:
    text = str(text)
    for punct in puncts + list(string.punctuation):
        if punct in text:
            text = text.replace(punct, f' {punct} ')
    #这里本身字符串为'你好!nihao?'，经过replace替换之后
    #对应的字符串为'你好 ! nihao ?' ,这里的{punct}内容实际上就是前面
    #标注的punct的内容，使用f正则表达式之后，中间的内容插入了空格
    return text


def clean_numbers(text: str) -> str:
    return re.sub(r'\d+', ' ', text)


def preprocess_text(text: str) -> str:
    text = text.lower()
    text = replace_typical_misspell(text)
    text = clean_text(text)
    text = clean_numbers(text)
    text = text.strip()
    #text.strip()去除字符串中相应的空格
    return text
#预处理措施很到位，去除标点符号，对应的时态以及相应的语态

In [6]:
text_columns = ['question', 'answer']
train_texts = train_df[text_columns].applymap(preprocess_text).values
#preprocess_text为上面刚刚定义过的函数内容
#applymap:将函数作用于DataFrame中的所有元素

test_texts = test_df[text_columns].applymap(preprocess_text).values
all_texts = list(itertools.chain(*train_texts, *test_texts))
#你想在多个对象执行相同的操作，但是这些对象在不同的容器中，
#你希望代码在不失可读性的情况下避免写重复的循环
#也就是说这里的train_texts和test_texts形成了一个共同的容器
#放在all_texts之中进行相同的操作

In [7]:
for  i  in  range(100):
    print(all_texts[i])

what am i losing when using extension tubes instead of a macro lens  ?   after playing around with macro photography on  -  the  -  cheap   (  read  :   reversed lens  ,   rev  .   lens mounted on a straight lens  ,   passive extension tubes  )    ,   i would like to get further with this  .   the problems with the techniques i used is that focus is manual and aperture control is problematic at best  .   this limited my setup to still subjects   (  read  :   dead insects  )   now  ,   as spring is approaching  ,   i want to be able to shoot live insects  .   i believe that for this  ,   autofocus and settable aperture will be of great help  .  

so  ,   one obvious but expensive option is a macro lens   (  say  ,   ef  mm macro  )   however  ,   i am not really interested in yet another prime lens  .   an alternative is the electrical extension tubes  .  

except for maximum focusing distance  ,   what am i losing when using tubes   (  coupled with a fine lens  ,   say ef   -     /    

In [8]:
def build_vocab(texts: List[str], max_features: int = 100000) -> Dict[str, Dict]:
    print('build_vocab !!!')
    counter = Counter()
    for text in texts:
        counter.update(text.split())
    
    filename = 'counter.txt'
    with  open(filename,'w') as file_object:
        file_object.write(str(counter))
    vocab = {}
    vocab['token2id'] = {
        token: _id + 1 for _id, (token, count) in
        enumerate(counter.most_common(max_features))}
    #counter保留单词相应出现的次数，
    #vocab['token2id']对于相应数值进行标号，
    #开头标1，接着标2，依次类推)
    #这里的_id+1就是为开头位置的'<PAD>'保留相应的位置
    
    #most_common:出现最多次数的词，counter.most_common(60000)
    #找出出现次数最多的60000个对应的单词
    vocab['token2id']['<PAD>'] = 0
    vocab['token2id']['<UNK>'] = len(vocab['token2id'])
    vocab['id2token'] = {v: k for k, v in vocab['token2id'].items()}
    vocab['word_freq'] = {
        **{'<PAD>': 0, '<UNK>': 0},
        **dict(counter.most_common(max_features)),
    }
    #这样操作的意义在于'<PAD>':0以及'<UNK>':0放在最前面
    #后面为接下来的数组依次往下走
    filename = 'vocab.txt'
    with  open(filename,'w')  as  file_object:
        file_object.write(str(vocab))
    return vocab
vocab = build_vocab(itertools.chain(*train_texts, *test_texts), max_features)
#最终vocab由3个部分组成 ['token2id'],['id2token'],['word_freq']三个部分组成

build_vocab !!!


python动态参数的两种形式:$*args$和$**kwarg$,这里面关键是一个和两个星号的区别，而不是args和kwargs在名字上的区别。
!!!注意:动态参数，必须放在所有的位置参数和默认参数后面!!!
~~~
def  func(name,age,sex='male',*args,**kwargs):
    pass
~~~
有时候我们传入一个列表，本意是希望将列表中的所有元素都当做参数传递进去，这里直接将$['a','b','c']$看做一个整体了，
怎么办?
其实只需要在调用时前面加一个$*$号，就能实现将列表中的每个元素传递进去了
~~~
def  func(*args):
    "表示接收任意个数量的参数，调用时会将实际参数
    打包为一个元组传入实参"
    for  i  in  args:
        print  i
        
func(*['a','b','c'])
参数依次放入
~~~

In [9]:
def tokenize(texts: List[str],
             token2id: Dict[str, int],
             max_len: int = 200) -> List[List[int]]:
    print('tokenize = !!!')
    def text2ids(text, token2id, max_len):
        return [
            token2id.get(token, len(token2id) - 1)
            for token in text.split()[:max_len]]
    #get(token,len(token2id)-1)中的后面一个内容len(token2id)-1为
    #对应的默认值
    #将text中的每一个单词进行切割分离，如果未出现的话对应内容为
    #len(token2id)-1
    #如果出现的话找寻出单词对应的id编号，传入的参数为vocab['token2id']
    
    tokenized = [
        text2ids(text, token2id, max_len)
        for text in texts]
    
    return tokenized


In [10]:
def load_embedding(embedding_path: str, word_index: Dict[str, int]) -> np.ndarray:
#embedding_path之中放入的是fasttext.pkl的对应的文件
    embeddings_index = joblib.load(embedding_path)
    #这个从路径中读出对应参数的操作很墨迹
    print('load_embedding = !!!')
    print('embeddings_index type = ')
    for  i,(k,v)  in  enumerate(embeddings_index.items()):
        if  i  in  range(0,10):
            print(k)
            print('###')
            print(v.shape)
            print('$$$')
        else:
            break
    #从fasttext.pkl中读出来的都是300维的向量，这点
    #很重要
    
    # word_index = tokenizer.word_index
    # 这里面的embedding_path对应的为相应的权重文本
    nb_words = min(max_features + 2, len(word_index))
    print('nb_words = %d'%nb_words)
    #nb_words = 46472
    embedding_matrix = np.zeros((nb_words, embed_size))
    for key, i in word_index.items():
        word = key
        embedding_vector = embeddings_index.get(word)
        #embeddings_index中获取word为得到相应的vector
        #数组对应内容
        
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = ps.stem(key)
        #ps = PorterStemmer()
        #将不同形式的单词转换为同一形式
        #比如create和created都得到creat
        #lc = LancasterStemmer()
        #
        #sb = SnowballStemmer('english')
        #这里面定义了三种对应的形式对单词进行切分
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        #分别通过不同的转化方式去提取出单词中的相应的矩阵
    #这里矩阵的内容返回的也是
    return embedding_matrix

def w2v_fine_tune(all_texts: List[str], vocab: Dict, embedding_matrix: np.ndarray) -> np.ndarray:
    print('w2v_fine_tune = !!!')
    model = Word2Vec(min_count=1, workers=1, iter=3, size=300)
    #min_count可以对字典做截断，词频少于min_count次数的单词会被
    #丢弃掉，默认值为5
    #workers:用于控制训练的并行数
    #size:是指特征向量的维度，默认值为100
    model.build_vocab_from_freq(vocab['word_freq'])
    #获取对应的词频字典(负采样优先选中词频较高的单词)
    idxmap = np.array(
        [vocab['token2id'][w] for w in model.wv.index2entity])
    #model.wv.index2entity(输出经过映射的词名称)
    #model.wv['sky']输出这个词的向量映射
    #这里面的'token2id'的对应内容为出现词的对应编号
    #频率高的编号,所以idxmap为所有在Word2Vec中出现过的词
    #对应的编号的内容
    
    model.wv.vectors[:] = embedding_matrix[idxmap]
    #model.wv.vectors的类型为ndarray类型
    print('model.wv.vectors = ***')
    print(model.wv.vectors.shape)
    #model.wv.vectors = (47460,300)
    #model.wv.vectors对应的是词向量
    print('***')
    model.trainables.syn1neg[:] = embedding_matrix[idxmap]
    #model.trainables.syn1neg = (47460,300)
    #注意这里数组为syn1neg,中间对应的数值为1!!!
    #model.trainables.syn1neg的类型为ndarray类型
    print('model.trainables.synlneg = ###')
    print(model.trainables.syn1neg.shape)
    #这里面的内容应该维syn1neg[:]的对应数组
    print('###')
    
    print('before train ***')
    print('model.wv.vectors = ***')
    print(model.wv.vectors[idxmap[0:10]])
    print('model.trainables.syn1neg = ***')
    print(model.trainables.syn1neg[idxmap[0:10]])
    print('###')
    #注意model.wv.vectors以及model.trainables.syn1neg这里初始化的时候
    #内容都为embedding_matrix中的矩阵
    model.train(all_texts, total_examples=len(all_texts), epochs=model.epochs)
    #!!!注意这里训练的all_texts的相应的内容既包括问句又包括答句
    #相当于问句和答句迭加在一起找词之间的相互关系
    print('after train ***')
    print('model.wv.vectors = ***')
    print(model.wv.vectors[idxmap[0:10]])
    print('model.trainables.syn1neg = ***')
    print(model.trainables.syn1neg[idxmap[0:10]])
    print('###')
    embedding_matrix = np.vstack([np.zeros((1, 300)), model.wv.vectors, np.zeros((1, 300))])
    #因为这里训练出来的内容为(1,300)维的向量，所以上面和下面都需要添加(1,300)维的向量
    #['<PAD>'] = 0,['<UNK>'] = len(vocab['token2id']),这里前后位置补充的是'<PAD>'以及
    #'<UNK>'的对应值
    
    #np.vstack():按垂直方向(行顺序)堆叠数组构成一个新的数组
    #堆叠的数组需要具有相同的维度
    
    #比如a = np.array([[1,2,3]]),b = np.array([[3,4,5]])
    #np.vstack((a,b))  
    #[[1 2 3]
    # [3 4 5]],np.hstack((a,b))为将数组对应横着叠加
    #[[1 2 3 3 4 5]]
    print('embedding_matrix.shape = ')
    print(embedding_matrix.shape)
    #embedding_matrix = (47462, 300)
    return embedding_matrix

In [11]:
vocab = build_vocab(itertools.chain(*train_texts, *test_texts), max_features)
#max_features = 60000
embedding_matrix = load_embedding(EMBEDDING_PATH, vocab['token2id'])
#此时embedding_matrix = (47462,300)的对应的矩阵
#!!!这里面先获得使用fasttext.pkl训练好的词向量矩阵作为初始化矩阵
embedding_matrix = w2v_fine_tune(all_texts, vocab, embedding_matrix)
#使用w2v_fine_tune 用word2vec对fasttext训练得到的结果进行操作

print('train_texts = ')
for  i  in  range(10):
    print(train_texts[i][0])
    print('##################################################################')
#print(train_texts[0:10,0])
print('train_texts1 = ')
for  i  in  range(10):
    print(train_texts[i][1])
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
train_q = tokenize(train_texts[:, 0], vocab['token2id'], max_q_len)
#这里[0]位置为question_title+question_body的内容,[1]位置为question_answer内容
#train_q = (6079,)
train_a = tokenize(train_texts[:, 1], vocab['token2id'], max_a_len)
#这里[1]位置为answer的对应的内容
#train_a = (6079,)
train_x = np.array([train_q, train_a]).T
#train_x = (6079,2)，一个问句以及一个答句
train_y = train_df[target_names].values
#train_y = (6079,30)
train_group = train_df['question_body'].values
#train_group之中存储'question_body'的对应内容
#train_group = (6079,)

test_q = tokenize(test_texts[:, 0], vocab['token2id'], max_q_len)
#test_q = (476,)
test_a = tokenize(test_texts[:, 1], vocab['token2id'], max_a_len)
#test_a = (476,)
test_x = np.array([test_q, test_a]).T
#test_x = (476,2)
#tokenize为前面定义过的函数，如果发现前面没有引用相关的库及包的
#时候，就需要考虑是不是这个函数已经被定义过了

#比如a=[1,2],b=[3,4],result = np.array([a,b]).T
#= [[1,2]   转置之后为 [[1,3]
#   [3,4]],             [2,4]]
#np.array为括号里面的数组按行依次放置

# target scaling
t_max = train_y.max(axis=0)[np.newaxis, :]
t_min = train_y.min(axis=0)[np.newaxis, :]
#train_y = train_df[target_names].values
#为对应属性的各项值
#axis = 1时，数组变化时横向的，体现出列的增加或减少
#axis = 0时，数组变化是纵向的，体现出行的增加或减少

#np.newaxis功能:插入新维度
#a = np.array([1,2,3,4,5])  shape = (5)
#aa = a[:,np.newaxis]  这里操作在后面插入一个新维度
#a = [[1],[2],[3],[4],[5]]  shape = (1,5)
#aa = a[np.newaxis,:]
#a = [[1 2 3 4 5]]   shape = (5,1)

#比如arr1 = np.array([[1,5,3],[4,2,6]])
#t_max = arr1.max(axis=0)[np.newaxis,:]操作之后
#t_max = [[4 5 6]]
#t_min = arr1.min(axis=0)[np.newaxis,:]操作之后
#t_min = [[1 2 3]]
print('t_max = ')
print(t_max)
print('t_min = ')
print(t_min)
train_y = (train_y - t_min) / (t_max - t_min)
#!!!训练完之后进行归一化操作很重要

build_vocab !!!
load_embedding = !!!
embeddings_index type = 
2000000
###
(300,)
$$$
,
###
(300,)
$$$
the
###
(300,)
$$$
.
###
(300,)
$$$
and
###
(300,)
$$$
to
###
(300,)
$$$
of
###
(300,)
$$$
a
###
(300,)
$$$
in
###
(300,)
$$$
is
###
(300,)
$$$
nb_words = 47462


I1219 00:30:22.926848 140155548452672 base_any2vec.py:974] Processing provided word frequencies
I1219 00:30:22.928339 140155548452672 base_any2vec.py:980] collected 47462 different raw word, with total frequency of 2602058
I1219 00:30:22.929272 140155548452672 word2vec.py:1647] Loading a fresh vocabulary
I1219 00:30:23.052832 140155548452672 word2vec.py:1671] effective_min_count=1 retains 47460 unique words (99% of original 47462, drops 2)
I1219 00:30:23.054460 140155548452672 word2vec.py:1677] effective_min_count=1 leaves 2602058 word corpus (100% of original 2602058, drops 0)


w2v_fine_tune = !!!


I1219 00:30:23.289059 140155548452672 word2vec.py:1736] deleting the raw counts dictionary of 47462 items
I1219 00:30:23.290625 140155548452672 word2vec.py:1739] sample=0.001 downsamples 53 most-common words
I1219 00:30:23.292421 140155548452672 word2vec.py:1742] downsampling leaves estimated 1804564 word corpus (69.4% of prior 2602058)
I1219 00:30:23.423434 140155548452672 base_any2vec.py:1022] estimated required memory for 47460 words and 300 dimensions: 137634000 bytes
I1219 00:30:23.425044 140155548452672 word2vec.py:1888] resetting layer weights
I1219 00:30:24.415891 140155548452672 base_any2vec.py:1210] training model with 1 workers on 47460 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5


model.wv.vectors = ***
(47460, 300)
***
model.trainables.synlneg = ###
(47460, 300)
###
before train ***
model.wv.vectors = ***
[[ 0.0231  0.017   0.0157 ...  0.0744 -0.1118  0.0963]
 [-0.0282 -0.0557 -0.0451 ... -0.037  -0.0725 -0.0042]
 [-0.0175 -0.2189  0.0353 ... -0.2846  0.0509  0.0229]
 ...
 [-0.062   0.1254  0.1693 ... -0.1727  0.0627 -0.0097]
 [ 0.0585  0.131  -0.0555 ...  0.1305 -0.0758  0.1905]
 [-0.2205  0.1434  0.038  ... -0.156   0.1814  0.0072]]
model.trainables.syn1neg = ***
[[ 0.0231  0.017   0.0157 ...  0.0744 -0.1118  0.0963]
 [-0.0282 -0.0557 -0.0451 ... -0.037  -0.0725 -0.0042]
 [-0.0175 -0.2189  0.0353 ... -0.2846  0.0509  0.0229]
 ...
 [-0.062   0.1254  0.1693 ... -0.1727  0.0627 -0.0097]
 [ 0.0585  0.131  -0.0555 ...  0.1305 -0.0758  0.1905]
 [-0.2205  0.1434  0.038  ... -0.156   0.1814  0.0072]]
###


I1219 00:30:25.424304 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 3.66% examples, 296337 words/s, in_qsize 1, out_qsize 0
I1219 00:30:26.435514 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.32% examples, 295491 words/s, in_qsize 1, out_qsize 0
I1219 00:30:27.450618 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 11.14% examples, 293888 words/s, in_qsize 2, out_qsize 0
I1219 00:30:28.460253 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 15.34% examples, 292836 words/s, in_qsize 1, out_qsize 0
I1219 00:30:29.462150 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 19.29% examples, 292138 words/s, in_qsize 2, out_qsize 0
I1219 00:30:30.466277 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 23.15% examples, 292754 words/s, in_qsize 2, out_qsize 0
I1219 00:30:31.478139 140155548452672 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 26.98% examples, 292855 words/s, in_qsize 1, out_qsize 0
I1219 00:30:32.486980 

I1219 00:31:22.766773 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 23.43% examples, 297543 words/s, in_qsize 2, out_qsize 0
I1219 00:31:23.773662 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 27.50% examples, 297843 words/s, in_qsize 2, out_qsize 0
I1219 00:31:24.776389 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 31.34% examples, 297939 words/s, in_qsize 1, out_qsize 0
I1219 00:31:25.782543 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 35.09% examples, 297675 words/s, in_qsize 2, out_qsize 0
I1219 00:31:26.794243 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 39.36% examples, 297698 words/s, in_qsize 1, out_qsize 0
I1219 00:31:27.809045 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 43.42% examples, 297694 words/s, in_qsize 1, out_qsize 0
I1219 00:31:28.811886 140155548452672 base_any2vec.py:1305] EPOCH 3 - PROGRESS: at 47.21% examples, 297711 words/s, in_qsize 1, out_qsize 0
I1219 00:31:29.82201

after train ***
model.wv.vectors = ***
[[ 0.0231      0.017       0.0157     ...  0.0744     -0.1118
   0.0963    ]
 [ 0.48918295 -0.11069084 -0.24687009 ...  0.2399514   1.0076635
   0.00949595]
 [-0.0175     -0.2189      0.0353     ... -0.2846      0.0509
   0.0229    ]
 ...
 [ 1.2186061   0.2925553   0.7809619  ... -1.1349126   0.70017743
  -0.99803126]
 [ 1.4345492  -0.99449205  0.30329013 ...  0.2699531  -0.18112485
   0.27332792]
 [-0.2205      0.1434      0.038      ... -0.156       0.1814
   0.0072    ]]
model.trainables.syn1neg = ***
[[ 0.0063947  -0.07385638  0.01361756 ...  0.06331608 -0.0991794
   0.07935498]
 [ 0.0479171  -0.14804259  0.00637321 ... -0.10388843  0.17515424
   0.01787014]
 [-0.0573205  -0.28986454  0.00773114 ... -0.2850605   0.07366499
   0.02994892]
 ...
 [ 0.10869126  0.08249416  0.04743605 ... -0.19042307  0.08605625
  -0.02372805]
 [ 0.10886309 -0.09930354 -0.14397602 ... -0.02560676  0.10888562
   0.00220892]
 [-0.23667435  0.06195231  0.02777862 ... 

tokenize = !!!
tokenize = !!!
tokenize = !!!
t_max = 
[[1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         0.66666667 1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.        ]]
t_min = 
[[0.33333333 0.33333333 0.         0.         0.         0.
  0.33333333 0.33333333 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.2        0.         0.         0.         0.33333333]]


上面先得到一个w2v训练出来的embedding_matrix的权重矩阵，然后下面又使用bert模型的文件得到另外一个相应的权重矩阵

In [12]:
class UniversalSentenceEncoder(object):
    
    def __init__(self, model_dir: str, batch_size: int = 128):
    #model_dir之中放置'universal-sentence-encoder-qa'的对应的目录
    #里面的内容含有saved_model.pb,以及variables的相应文件夹，variables
    #之中含有variables.data-00000-of-00001以及variables.index
    #这两个对应的相应的文件
        self.module = hub.load(str(model_dir))
    #这里面的load函数将对应的variables.data-00000-of-00001
    #以及variables.index两个对应文件同时加载出来
    #hub.load这里是载入模型，所以载入的是一个目录
    
    #import tensorflow_hub as hub
    #迁移学习是把已训练好的模型(预训练模型)参数迁移到新的模型
    #来帮助新模型训练。
    
    #实现迁移学习的手段
    #1.Transfer Learning:冻结预训练模型的全部卷积层，只训练自己定制
    #的全连接层
    #2.Extract Feature Vector:先计算出预训练模型的卷积层对所有训练
    #和测试数据的特征向量，然后抛开预训练模型，只训练自己定制的简配版全连接网络。
    #3.Fine-Tuning:冻结预训练模型的部分卷积层（通常是靠近输入的多数卷积层，因为这
    #些层保留了大量底层信息）甚至不冻结任何网络层，训练剩下的卷积层（通常是靠近输
    #出的部分卷积层）和全连接层。
        self.batch_size = batch_size
    #二进制文件中包含所有的weights,biases,gradients和其他的variables的值
    #这个文件使用.ckpt后缀，有两个对应的文件
    #variables.data-00000-of-00001以及variables.index,.data文件就是保存
    #训练的variables
    
    #所有预训练之后的文件都放在相应的文件夹之中，比如saved_model.pd
    #variables之中的variables.data-00000-of-00001,variables.index的文件
        
    def __call__(self, texts: List[str], mode: str) -> torch.FloatTensor:
        assert mode in ['question', 'answer']
        embeddings = []
        for i in range(0, len(texts), self.batch_size):
            text = texts[i:(i + self.batch_size)]
            #提取texts[i:(i+self.batch_size)]中的文本信息
            if mode == 'question':
        #TensorFlow Hub是一个用于促进机器学习模型中可复用部分再次进行
        #探索与发布的库，主要是预训练过的Tensorflow模型片段再次利用到
        #新的任务上
                h_embedding = self.module.signatures['question_encoder'](
                    tf.constant(text))['outputs']
            #self.module = hub.load(str(model_dir))
            else:
                h_embedding = self.module.signatures['response_encoder'](
                    input=tf.constant(text),
                    context=tf.constant(text))['outputs']
                
            h_embedding = torch.FloatTensor(h_embedding.numpy())
            embeddings.append(h_embedding)
        return torch.cat(embeddings, 0)

In [ ]:
print(train_df['question'][0])
print(test_df['question'][0])

In [13]:
use = UniversalSentenceEncoder(USE_DIR, batch_size=1)
q_texts = list(itertools.chain(train_df['question'].values, test_df['question'].values))
print(q_texts[0])
a_texts = list(itertools.chain(train_df['answer'].values, test_df['answer'].values))
print(a_texts[0])
#这里面q_texts以及a_texts都是文本，
q_emb = use(q_texts, mode='question')
#使用use函数将问题对应的权重矩阵提取出来
#q_emb = (6555,512),本身len(train_df) = 6079,len(a_emb) = 476,所以len(q_emb) = 6555
a_emb = use(a_texts, mode='answer')
#a_emb = (6555,512),里面隐藏着self.batch_size = 1
print('len(train_df) = %d'%len(train_df))
train_q_emb, test_q_emb = q_emb[:len(train_df)], q_emb[len(train_df):]
train_a_emb, test_a_emb = a_emb[:len(train_df)], a_emb[len(train_df):]
#从itertool.chain中截取出train_q_emb以及test_q_emb中相应内容
print(np.array(train_q_emb).shape)
#train_q_emb = (6079,512)
print(np.array(test_q_emb).shape)
#test_q_emb = (476,512)
print(np.array(train_a_emb).shape)
#train_a_emb = (6079,512)
print(np.array(test_a_emb).shape)
#test_q_emb = (476,512)

I1219 01:06:51.981954 140155548452672 registry.py:53] resolver HttpCompressedFileResolver does not support the provided handle.
I1219 01:06:51.983100 140155548452672 registry.py:53] resolver GcsCompressedFileResolver does not support the provided handle.
I1219 01:06:51.983887 140155548452672 registry.py:53] resolver HttpUncompressedFileResolver does not support the provided handle.


What am I losing when using extension tubes instead of a macro lens? After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.

So, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am not really interested in yet another prime lens. An alternative is the electrical extension tubes.

Except for maximum focusing distance, what am I losing when using tubes (coupled with a fine lens, say EF70-200/2.8) instead of a macro lens?

I just got extension tubes, so here's the skinny.


  ...what am I losing when using tubes...?

In [14]:
print('q_emb.shape = ')
print(np.array(q_emb).shape)
#q_emb = (6555,512)
print('a_emb.shape = ')
print(np.array(a_emb).shape)
#a_emb = (6555,512)
print('train_df.shape = ')
print(np.array(train_df).shape)
#train_df = (6079,42)
print('q_emb = ')
print(q_emb)
#q_emb为一个6555*512的权重矩阵
print('a_emb = ')
print(a_emb)
#a_emb为一个6555

#这里训练出来的对应内容后面才会用到

q_emb.shape = 
(6555, 512)
a_emb.shape = 
(6555, 512)
train_df.shape = 
(6079, 42)
q_emb = 
tensor([[ 0.0474, -0.0153,  0.0554,  ...,  0.0042,  0.0046,  0.0418],
        [ 0.0633, -0.0249,  0.0011,  ..., -0.0287, -0.0181,  0.0133],
        [-0.0070,  0.0484, -0.0083,  ...,  0.0430,  0.0517,  0.0322],
        ...,
        [ 0.0613,  0.0327, -0.0353,  ...,  0.0827, -0.0024, -0.0913],
        [ 0.0152,  0.0706, -0.0084,  ...,  0.0548,  0.0321,  0.0410],
        [-0.0577, -0.0303, -0.0383,  ...,  0.0382, -0.0041, -0.0004]])
a_emb = 
tensor([[ 0.0363, -0.0201,  0.0728,  ..., -0.0221,  0.0135, -0.0279],
        [ 0.0477, -0.0742, -0.0025,  ..., -0.0641,  0.0107, -0.0709],
        [-0.0482,  0.0220,  0.0407,  ..., -0.0528,  0.0384,  0.0372],
        ...,
        [ 0.0518, -0.0172, -0.0412,  ...,  0.0423, -0.0334, -0.1007],
        [-0.0169,  0.0823, -0.0198,  ...,  0.0356,  0.0324, -0.0193],
        [-0.0567, -0.0106, -0.1018,  ..., -0.0576, -0.0535,  0.0446]])


In [16]:
train_df

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,question
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000,What am I losing when using extension tubes in...
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.555556,0.888889,0.888889,0.666667,0.000000,0.000000,0.666667,0.888889,What is the distinction between a city and a s...
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.555556,1.000000,1.000000,0.666667,0.000000,0.333333,1.000000,0.888889,Maximum protusion length for through-hole comp...
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.833333,0.333333,0.833333,1.000000,0.800000,0.000000,0.000000,1.000000,1.000000,Can an affidavit be used in Beit Din? An affid...
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,1.000000,1.000000,How do you make a binary image in Photoshop? I...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,sixtyfootersdude,https://bicycles.stackexchange.com/users/134,If you're thinking about wearing a ski helmet ...,Matt Leo,https://bicycles.stackexchange.com/users/3340,http://bicycles.stackexchange.com/questions/99...,CULTURE,...,1.000000,0.555556,1.000000,1.000000,0.866667,0.000000,0.000000,0.000000,0.888889,Using a ski helmet for winter biking I am curi...
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,ash,https://bicycles.stackexchange.com/users/14519,\nYou can replace the pads (as stated elsewher...,Daniel R Hicks,https://bicycles.stackexchange.com/users/1584,http://bicycles.stackexchange.com/questions/25...,CULTURE,...,0.888889,0.555556,1.000000,1.000000,0.733333,0.666667,0.333333,0.000000,0.888889,Adjustment to road bike brakes for high grade ...
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maneating Koala,https://unix.stackexchange.com/users/60445,Maybe help if can be fixes origin of this erro...,BG Bruno,https://unix.stackexchange.com/users/68208,http://unix.stackexchange.com/questions/169054...,TECHNOLOGY,...,0.888889,0.555556,0.888889,0.888889,0.80

q_emb = 
tensor([[ 0.0474, -0.0153,  0.0554,  ...,  0.0042,  0.0046,  0.0418],
        [ 0.0633, -0.0249,  0.0011,  ..., -0.0287, -0.0181,  0.0133],
        [-0.0070,  0.0484, -0.0083,  ...,  0.0430,  0.0517,  0.0322],
        ...,
        [ 0.0613,  0.0327, -0.0353,  ...,  0.0827, -0.0024, -0.0913],
        [ 0.0152,  0.0706, -0.0084,  ...,  0.0548,  0.0321,  0.0410],
        [-0.0577, -0.0303, -0.0383,  ...,  0.0382, -0.0041, -0.0004]])
a_emb = 
tensor([[ 0.0363, -0.0201,  0.0728,  ..., -0.0221,  0.0135, -0.0279],
        [ 0.0477, -0.0742, -0.0025,  ..., -0.0641,  0.0107, -0.0709],
        [-0.0482,  0.0220,  0.0407,  ..., -0.0528,  0.0384,  0.0372],
        ...,
        [ 0.0518, -0.0172, -0.0412,  ...,  0.0423, -0.0334, -0.1007],
        [-0.0169,  0.0823, -0.0198,  ...,  0.0356,  0.0324, -0.0193],
        [-0.0567, -0.0106, -0.1018,  ..., -0.0576, -0.0535,  0.0446]])

In [17]:
train_text_emb = torch.cat((train_q_emb, train_a_emb), 1)
#原先train_q_emb = [6079,512],train_a_emb = [6079,512],
#拼接完之后train_text_emb = [6079,1024]
test_text_emb = torch.cat((test_q_emb, test_a_emb), 1)
#torch.cat(,)后面值为0的时候表示按维数0(行进行拼接)
#比如A = tensor([[1,1,1],[1,1,1]]),B = tensor([[2,2,2],[2,2,2]])
#C = torch.cat((A,B),0)之后
#C = tensor([[1,1,1],
#             [1,1,1],
#             [2,2,2],
#             [2,2,2]])
#如果A = tensor([[1,1,1],[1,1,1]]),B = tensor([[2,2,2,2],[2,2,2,2]])
#    C = tensor([[1,1,1,2,2,2,2],
#                [1,1,1,2,2,2,2]])
#test_text_emb = 

#train_q_emb+train_a_emb
#test_q_emb+test_a_emb
del use
cuda.select_device(0)
#假设有4块显卡，编号分别为0,1,2,3
#如果我们想使用第0到n块卡，只需要进行相应的设置即可
cuda.close()

I1219 01:30:44.995497 140155548452672 driver.py:224] init
I1219 01:30:44.999042 140155548452672 driver.py:603] reset context of device 0
I1219 01:30:45.000501 140155548452672 driver.py:605] reset context of device 0


In [18]:
print('train_text_emb.shape = ')
print(train_text_emb.shape)
print('test_text_emb.shape = ')
print(test_text_emb.shape)

train_text_emb.shape = 
torch.Size([6079, 1024])
test_text_emb.shape = 
torch.Size([476, 1024])


In [19]:
class TextDataset(Dataset):

    def __init__(self, seqs, targets=None):
    #第一次传入的是一问一答，seqs是问句,targets是答句
    #第二次传入的时候只传入问句，注意看__getitem__之中的
    #内容，取出很有艺术感，一个类将两种取出方式都搞定了
        print('TextDataset __init__')
        self.seqs = seqs
    #使用self.seqs = seqs,这里的seqs对应着
    #相应的test_x([476,2])的问答构成的矩阵
        self.targets = targets
    #这里面self.targets构成矩阵的特点
        
    def __len__(self):
        print('TextDataset __len__')
    #使用__len__限定循环的最大长度
        return len(self.seqs)
        
    def get_keys(self):
        print('TextDataset get_keys')
        #print('before self.seqs')
        #self.seqs.shape = (476,2)
        #for  i  in  range(len(self.seqs)):
        #    print('i = %d:%d,%d'%(i,len(self.seqs[i][0]),len(self.seqs[i][1])))
        #self.seqs[i][0] = 225,self.seqs[i][1] = 28,此时
        #get_keys(self)输出的第一个维度为225+28=253
        return np.vectorize(len)(self.seqs).sum(axis=1)
    #axis = 1就是将矩阵的从左到右的向量取出最大值，这里对应的向量里面的内容为self.seqs
    #注意千万不要用行和列的思想去想axis，因为行和列是没有方向的，这样想会在遇到不同的
    #例子时感到困惑
    
    def __getitem__(self, index):
        #print('TextDataset __getitem__')
        if self.targets is None:
            return index, self.seqs[index]
        return index, self.seqs[index], self.targets[index]
    #很有艺术感的取出方式

def collate_fn(data):
    #感觉这里的collate_fn是对相应的数据进行处理
    print('function collate_fn')
    def _pad_sequences(seqs):
        print('collate_fn _pad_sequences')
        lens = [len(seq) for seq in seqs]
        max_len = max(lens)

        padded_seqs = torch.zeros(len(seqs), max_len).long()
        for i, seq in enumerate(seqs):
            start = max_len - lens[i]
            padded_seqs[i, start:] = torch.LongTensor(seq)
        return padded_seqs

    transposed = list(zip(*data))
    index = transposed[0]
    q_seqs, a_seqs = zip(*transposed[1])
    q_seqs = _pad_sequences(q_seqs)
    a_seqs = _pad_sequences(a_seqs)
    seqs = [q_seqs, a_seqs]
    if len(transposed) == 2:  # targets == None
        return index, seqs
    return index, seqs, torch.FloatTensor(transposed[2])


class BucketSampler(Sampler):
#这里面相当于自定义了Sampler
    def __init__(self, data_source, sort_keys, bucket_size=None, batch_size=1048, shuffle_data=True):
        super().__init__(data_source)
    #test_sampler = BucketSampler(test_dataset,test_dataset.get_keys(),batch_size = batch_size,
    #shuffle = False),这里面定义的batch_size = 128
    
    #后面的循环之中的定义
    #第一次循环的时候
    #train_sampler = BucketSampler(train_dataset, train_dataset.get_keys(),
    #                              bucket_size=batch_size * 20, batch_size=batch_size)
    #bucket_size = 128*20 = 2560,batch_size = 128
    #第二次循环的时候
    #valid_sampler = BucketSampler(valid_dataset, valid_dataset.get_keys(),
    #                              batch_size=batch_size, shuffle_data=False)
    #这里面定义的data_source = test_dataset,sort_keys = test_dataset.get_keys(),batch_size = 
    #shuffle = False，这里面的操作中batch_size = 128
        print('BucketSampler __init__')
        self.shuffle = shuffle_data
        #传入待检验的对应的数据
        
        #传入False
        self.batch_size = batch_size
        #batch_size = 128
        self.sort_keys = sort_keys
        #调入的参数为test_dataset.get_keys(),这之后sort_keys = (476,)
        print('after get_keys')
        print(sort_keys)
        print('sort_keys.shape = ')
        print(np.array(sort_keys).shape)
        self.bucket_size = bucket_size if bucket_size is not None else len(sort_keys)
        print('self.bucket_size = ')
        print(self.bucket_size)
        #self.bucket_size = 128*20 = 2560
        #这里面传入的bucket_size = None,所以这里bucket_size变为476
        self.weights = None

        if not shuffle_data:
        #这里shuffle_data = False,正好调用的prepare_buckets()
            self.index = self.prepare_buckets()
            print('!!!!!!!!!!!!!!!!!!!!!self.index = !!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(self.index)
        else:
            self.index = None
        #这里面返回的self.index = [132,274,146,170,...,408,74,82]
        #一顿操作猛如虎，一看战绩0-5，这里面的self.index的值与self.sort_keys
        #的对应值一样

    def set_weights(self, weights):
        assert weights >= 0
        print('BucketSampler set_weights')
        total = np.sum(weights)
        if total != 1:
            weights = weights / total
        self.weights = weights

    def __iter__(self):
        print('BucketSampler __iter__')
        indices = None
        if self.weights is not None:
            total = len(self.sort_keys)
            indices = np.random.choice(total, (total,), p=self.weights)
        if self.shuffle:
            self.index = self.prepare_buckets(indices)
        return iter(self.index)

    def get_reverse_indexes(self):
        print('BucketSampler get_reverse_indexes')
        indexes = np.zeros((len(self.index),), dtype=np.int32)
        for i, j in enumerate(self.index):
            indexes[j] = i
        return indexes

    def __len__(self):
        print('BucketSampler __len__')
        return len(self.sort_keys)
        
    def prepare_buckets(self, indices=None):
        print('BucketSampler prepare_buckets')
        lens = - self.sort_keys
        #self.sort_keys为之前的(476,)的相应矩阵，用来判断len(self.sort_keys[0])
        #以及len(self.sort_keys[1])的相应的长度的，
        assert self.bucket_size % self.batch_size == 0 or self.bucket_size == len(lens)
        #self.bucket_size = 476,self.batch_size = 128
        
        if indices is None:
        #传入indices = None
            if self.shuffle:
            #传入self.shuffle = False，
                indices = shuffle(np.arange(len(lens), dtype=np.int32))
            #这里shuffle为from sklearn.util import shuffle
            #lens = (476,)
                lens = lens[indices]
            else:
            #这里传入的self.shuffle = False,由于初始化的时候传入的self.shuffle
            #= False的时候才调用prepare_buckets,所以只能调用这一个else之中的内容
                indices = np.arange(len(lens), dtype=np.int32)
        print('indices = ')
        print(indices)
        #indices为[0,1,2,...475]构成的相应数组
        
        #  bucket iterator
        def divide_chunks(l, n):
            print('BucketSampler divide_chunks')
            if n == len(l):
                yield np.arange(len(l), dtype=np.int32), l
            else:
                # looping till length l
                for i in range(0, len(l), n):
                    data = l[i:i + n]
                    yield np.arange(i, i + len(data), dtype=np.int32), data
                #这里需要明白yield函数的原理，每次运行的时候接着上次的进行运行
        new_indices = []
        extra_batch = None
        print('lens = ')
        print(lens)
        print('self.bucket_size = ')
        print(self.bucket_size)
        #lens = [-253 -226 -203 -876...
        #       .....................]
        #lens对应的数值为原先sort_keys()中的数组取对应的负值
        for chunk_index, chunk in divide_chunks(lens, self.bucket_size):
            # sort indices in bucket by descending order of length
            #lens = [-253 -226 -203 -876...],bucket_size = 476
            
            #chunk_index = [0 1 2 3 4 ... 475],chunk = [-253 -226 -203...-277]
            #这里的chunk为之前sort_keys相加之后取的负数
            #因为len(lens) == self.bucket_size,所以这里总共就一波
            print('chunk_index = ')
            print(chunk_index)
            print('chunk = ')
            print(chunk)
            indices_sorted = chunk_index[np.argsort(chunk, axis=-1)]
            #np.argsort返回排序后的索引值的数组，
            print('indices_sorted = ')
            print(indices_sorted)
            #indices_sorted = [132,274,146,170...144,240,172]
            print('chunk 132 = ')
            print(chunk[132])
            batches = []
            for u, batch in divide_chunks(indices_sorted, self.batch_size):
                print('u = ')
                print(u)
                #第一次的时候u = [0,1,2,3,4...127]
                #第二次的时候u = [128,129,...255]
                #第三次的时候u = [256,257,258...383]
                #第四次的时候u = [384,385,...474,475]
                print('batch = ')
                print(batch)
                #第一次的时候batch = [132,274,146,170...144,240,172]
                #batch = 128
                #这里chunk[132] = -1024
                #第二次的时候batch = []
                if len(batch) == self.batch_size:
                    batches.append(batch.tolist())
                else:
                    assert extra_batch is None
                    assert batch is not None
                    extra_batch = batch
            #记录最后一组对应的batch内容
            # shuffling batches within buckets
            #这里对应的self.shuffle = False
            if self.shuffle:
                batches = shuffle(batches)
            #它这个传入的参数之中self.shuffle = False,一直不准备打乱
            #相应的数值
            
            #它这个每组取出来的操作感觉是将相应的数组内容打乱
            #但是这里面它进行排序了，所以就没有想进行这个操作
            print('***************************batches = ********************************')
            print(batches)
            print('*********************************************************************')
            for batch in batches:
                new_indices.extend(batch)
            print('new_indices = ')
            print(new_indices)
            #new_indices为对应的上面和的长度最大的放在前面，最小的放在后面
            #(实际上是和的负值的最小放在前面，最大放在后面)
    
        if extra_batch is not None:
            new_indices.extend(extra_batch)
        #new_indices当中先放入相应的batch的内容，再放入相应的extra_batch
        #的内容，注意是往里面不断填充的
        
        #!!!这里面的操作是傻逼操作，一顿操作猛如虎，回头一看0-5，操作了半天
        #之后new_indices的内容为[132, 274, 146, 170, 351, 72, 210, 457, 57, 100,
        #...303,408,74,82]
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!new_indices = !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(len(new_indices))
        print('#############################indices = ##################################')
        print(indices)
        print('###############')
        #indices = [0,1,2,3,4,....471,472,473,474,475]
        #new_indices = [132,274,146]
        return indices[new_indices]

类与类定义在一起，运行过程与运行过程定义在一起的好处就是运行的时候发现类中有需要添加或者删除的打印信息的时候，
可以只修改相应的类模块，再通过运行看对应的类模块

In [20]:
test_dataset = TextDataset(test_x)
#将test_x放入TextDataset之中，test_x = (476,2),里面的2中的第一个list
#内容为第一个list对应值([53, 1860, 13032, 4097, 328, 7199, 49, 18435, 32, 12, 131,
#363, 13, 293, 817, 102, 63, 4, 72, 2276, 16, 13032, 45, 64, 58, 31, 
#250, 59, 2, 4646, 4, 332, 117, 4097, 328, 40, 1, 12, 122, 25, 12, 42, 
#67, 8, 30598, 4, 495, 117, 15, 25, 76, 53, 1869, 37, 1087, 732, 19, 8, 45595, 1...])
#(里面的内容为问句的单词相应的编号)
#第二个list对应值 list([57, 11, 95, 6243, 30, 1860, 13032, 1719, 1, 71, 23, 79, 38, 
#1098, 8, 1394, 16, 1845, 19, 55, 342, 16, 508, 8, 1511, 3170, 7076, 1])]
#(里面的内容为答句的单词相应的编号)
test_sampler = BucketSampler(test_dataset, test_dataset.get_keys(),
                             batch_size=batch_size, shuffle_data=False)
#这里面定义完test_sampler之后就初始化完了

#get_keys()输入的是两个数组长度相加，比如self.seqs[0][0] = 225,self.seqs[0][1] = 28
#此时得到的长度self.seqs[0] = 225+28 = 253
print('collate_fn = ')
print(collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler,
                         shuffle=False, num_workers=0, collate_fn=collate_fn)
#DataLoader本质上是一个可迭代的对象，使用iter()访问，不能使用next()访问
#collate_fn应当是一个可调用对象，DataLoader不设置collate_fn参数时每个
#mini-batch输出的样本序列，所以可以理解为一个后处理的函数

#class BucketSampler(Sampler)这里定义BucketSampler的时候直接继承了Sampler的内容
#collate_fn为上面定义了的相应的collate_fn的函数



TextDataset __init__
TextDataset get_keys
BucketSampler __init__
after get_keys
[ 253  226  203  876  211  117  689  320  466  111  724  101  593  175
  257  684  560  190  365  219  111  541  120  131   92  368  709  347
  349  433  292  817  333  149  299  547  174  714  266  206  305  723
  579  650  817  300  241  305  613  162  236  240  765  495  452  151
  103  983  285  424  187  201   95  183  256  262  171  622  148  216
  265  310 1011  273  167  121  339  232  179  196  261  429  167  294
   70  383  615  435  211  225  302  138   70  310   70  725  148  628
  139  441  965  491  197  169  233   72  213  134  400  294  559  182
  514  145  183  738  553  416  389  171  347  286  732  372   82  712
   87  366  110  425  126  502 1024  386  224  308  567  253  608  405
  582  578  365  345   68  310 1024  344  112  938  362  297  588  125
  225  147  145  517  135  211  570  202  379  293  701  251  344  206
  286  246 1024  157   52  155  281  393  180  204  430  296  331  1

In [21]:
print(test_sampler)

In [23]:
class LstmUnit(nn.Module):
    
    def __init__(self, embedding_matrix, lstm_hidden_size=120, gru_hidden_size=60):
        super(LstmUnit, self).__init__()
        print('LstmUnit __init__')
    #第一次初始化的时候self.q_lstm = LstmUnit(embedding_matrix, q_lstm_size(120), q_lstm2_size(120))
    #第二次初始化的时候self.a_lstm = LstmUnit(embedding_matrix, a_lstm_size(120), a_lstm2_size(120))
        self.embedding = nn.Embedding(*embedding_matrix.shape)
    #*args与**kwargs的区别：两者都是python中的可变参数
    #这里面的embedding_matrix.shape正好为一个矩阵的形状数组，所以使用*
    #标志进行num_embeddings以及embedding_dim的数值放置
    
    #*args表示任何多个无名参数，它本质上是一个tuple
    #**kwargs表示关键字参数，它本质上是一个dict
    
    #torch.nn.Embedding(num_embeddings,embedding_dim,padding_idx = None,
    #max_norm = None,norm_type = 2,scale_grad_by_freq = False,sparse = False)

    #如果同时使用*args和**kwargs时，必须*args参数列要在**kwargs之前
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
    #在pytorch里面使用word2vec训练好的词向量,embedding_matrix = (47462,300)
        self.embedding.weight.requires_grad = False
    #在反向传播的时候，不要对这些词向量进行求导更新
        self.embedding_dropout = nn.Dropout2d(0.2)

        print('embedding_matrix.shape1 = ')
        print(embedding_matrix.shape[1])
        self.lstm = nn.LSTM(embedding_matrix.shape[1], lstm_hidden_size, bidirectional=True, batch_first=True)
        #nn.LSTM输入的参数列表表示:input_size输入数据的特征维数，通常就是
        #embedding_dim(词向量的维度) = 300,lstm_hidden_size = 120
        
        #hidden_size = lstm_hidden_size为LSTM中隐层的维度
        #num_layers:循环神经网络的层数，bias:用不用偏置，default = True
        
        #batch_first 这个要注意，通常我们输入的数据shape=(batch_size,seq_length,embedding_dim),而batch_first默认
        #是False,所以我们的输入数据最好送进LSTM之前将batch_size与seq_length这两个维度调换
        #如果batch_first = True的时候，此时第一个放入batch_size(embedding_matrix.shape[1])即可
        #因为此时batch_first意味着batch_size放在第一位的位置
        
        #dropout:默认是0，代表不用dropout
        #bidirectional默认是false，代表不用双向LSTM
        self.lstm2 = nn.LSTM(lstm_hidden_size * 2, gru_hidden_size, bidirectional=True, batch_first=True)
        #lstm_hidden_size*2 = 240,gru_hidden_size = 60
        
    def apply_spatial_dropout(self, h_embedding):
        print('LstmUnit apply_spatial_dropout')
        h_embedding = h_embedding.transpose(1, 2).unsqueeze(2)
        print('shape1 = ')
        print(h_embedding.transpose(1,2).shape)
        print('shape2 = ')
        print(h_embedding.transpose(1,2).unsqueeze(2).shape)
        h_embedding = self.embedding_dropout(h_embedding).squeeze(2).transpose(1, 2)
        return h_embedding
    
    def flatten_parameters(self):
        print('LstmUnit flatten_parameters')
        self.lstm.flatten_parameters()
        print('self.lstm.shape = ')
        print(self.lstm.shape)
        self.lstm2.flatten_parameters()
        print('self.lstm2.shape = ')
        print(self.lstm2.shape)
        
    def forward(self, x):
        print('LstmUnit forward')
        batch_size = x.size(0)
        h_embedding = self.embedding(x)
        #self.embedding = nn.Embedding(*embedding_matrix.shape)
        print('h_embedding1 = ')
        print(h_embedding.shape)
        h_embedding = self.apply_spatial_dropout(h_embedding)
        print('h_embedding2 = ')
        print(h_embedding.shape)
        h_lstm, data1 = self.lstm(h_embedding)
        print('h_lstm = ')
        print(h_lstm)
        print('data1 = ')
        print(data1)
        #self.lstm = nn.LSTM(embedding_matrix.shape[1], lstm_hidden_size, bidirectional=True, batch_first=True)
        h_lstm, data2 = self.lstm2(h_lstm)
        print('h_lstm = ')
        print(h_lstm)
        print('data2 = ')
        print(data2)
        #self.lstm2 = nn.LSTM(lstm_hidden_size * 2, gru_hidden_size, bidirectional=True, batch_first=True)

        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)

        out = torch.cat((avg_pool, max_pool), 1)
        return out


class LstmModel(nn.Module):
    
    def __init__(self, embedding_matrix):
        super(LstmModel, self).__init__()
        print('LstmModel __init__')
        #embedding_matrix为上面fasttext+word2vec训练出来的内容
        #
        
        q_lstm_size = 120
        q_lstm2_size = 120
        
        a_lstm_size = 120
        a_lstm2_size = 120
        
        self.q_lstm = LstmUnit(embedding_matrix, q_lstm_size, q_lstm2_size)
        self.a_lstm = LstmUnit(embedding_matrix, a_lstm_size, a_lstm2_size)

        self.linear = nn.Linear((q_lstm2_size + a_lstm2_size) * 4 + 512 * 2, 200)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(200, num_targets)
        
    def flatten_parameters(self):
        print('LstmModel flatten_parameters')
        self.q_lstm.flatten_parameters()
        self.a_lstm.flatten_parameters()
        
    def forward(self, q_seqs, a_seqs, text_emb):
        print('LstmModel forward')
        h_q = self.q_lstm(q_seqs)
        h_a = self.a_lstm(a_seqs)

        conc = torch.cat((h_q, h_a, text_emb), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)

        return out


In [ ]:
class EMA(object):

    def __init__(self, model, mu, level='batch', n=1):
        # self.ema_model = copy.deepcopy(model)
        print('EMA __init__')
        self.mu = mu
        self.level = level
        self.n = n
        self.cnt = self.n
        self.shadow = {}
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data

    def _update(self, model):
        print('EMA _update')
        for name, param in model.named_parameters():
            if param.requires_grad:
                new_average = (1 - self.mu) * param.data + self.mu * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def set_weights(self, ema_model):
        print('EMA set_weights')
        for name, param in ema_model.named_parameters():
            if param.requires_grad:
                param.data = self.shadow[name]

    def on_batch_end(self, model):
        print('EMA on_batch_end')
        if self.level is 'batch':
            self.cnt -= 1
            if self.cnt == 0:
                self._update(model)
                self.cnt = self.n
                
    def on_epoch_end(self, model):
        print('EMA on_epoch_end')
        if self.level is 'epoch':
            self._update(model)
def get_scores(y_true, y_pred) -> Dict[str, float]:
    print('get_scores')
    # y_true, y_pred: np.ndarray with shape (sample_size, num_targets)
    assert y_true.shape == y_pred.shape
    assert y_true.shape[1] == num_targets
    scores = {}
    for target_name, i in zip(target_names, range(y_true.shape[1])):
        scores[target_name] = scipy.stats.spearmanr(y_true[:, i], y_pred[:, i])[0]
    return scores


def predict(model: nn.Module,
            data_loader: DataLoader,
            text_emb: torch.Tensor,
            device: torch.device = torch.device('cuda')) -> np.ndarray:
    print('predict')
    model.eval()
    preds_fold = np.zeros((len(data_loader.dataset), num_targets))

    with torch.no_grad():
        for index, x_batch in data_loader:
            x_batch = (x.to(device) for x in x_batch)
            emb_batch = text_emb[list(index)].to(device)
            y_pred = model(*x_batch, emb_batch).detach()
            preds_fold[list(index)] = torch.sigmoid(y_pred.cpu()).numpy()
    return preds_fold


In [24]:
gkf = GroupKFold(n_splits=n_splits)
cv_scores = []
ema_cv_scores = []
fold_scores = []
train_preds = np.zeros((len(train_x), num_targets))
#train_preds = (6079,30)
test_preds = np.zeros((len(test_x), num_targets))
#test_preds = (476,30)
ema_train_preds = np.zeros((len(train_x), num_targets))
#ema_train_preds = (6079,30)
ema_test_preds = np.zeros((len(test_x), num_targets))
#ema_test_preds = (476,30)

#输入输出的内容太多的时候，将对应的打印内容提取到一个相应的文件上
#然后一句一句分析其中相应的内容
for i, (train_idx, valid_idx) in enumerate(list(gkf.split(train_x, train_y, train_group))):
    #train_x = (6079,2):对应内容为问句+答句，train_y = (6079,30):30个内容的标签
    #gkf.split(train_x,train_y,train_group):仅将训练集进行划分
    #train_group:Always ignored,exists for compatibility
    print(f'fold {i + 1}')
    #(train_idx,valid_idx) = (train_x,train_y)
    #打印出fold {i+1} = fold {0+1} = fold {1}
    train_fold_x, train_fold_y = train_x[train_idx], train_y[train_idx]
    valid_fold_x, valid_fold_y = train_x[valid_idx], train_y[valid_idx]
    #train_x = np.array([train_q,train_a]).T,train_x = (6079,2),对应一个问句和一个答句
    #这里切出来的train_idx为训练集，valid_idx为验证集，train_y = (6079,30)
    
    train_fold_emb = train_text_emb[train_idx]
    valid_fold_emb = train_text_emb[valid_idx]
    #train_fold_emb = train_text_emb[train_idx]
    #valid_fold_emb = train_text_emb[valid_idx]
    print('train_fold_emb = ')
    print(train_fold_emb.shape)
    #train_fold_emb = [4863,1024]
    print('valid_fold_emb = ')
    print(valid_fold_emb.shape)
    #train_fold_emb = [4863,1024]
    #valid_fold_emb = [1216,1024]
    #问句和答句融合在一起的对应的内容
    train_dataset = TextDataset(train_fold_x, train_fold_y)
    #这里面TextDataset初始化定义为def __init__(self, seqs, targets=None):
    #如果同时出现问句和答句的话，使用TextDataset将问句和答句的长度加在一起
    #构成相应的数组
    valid_dataset = TextDataset(valid_fold_x)
    #如果只出现一个句子的话，使用TextDataset算出相应的valid_fold_x的内容
    #选出的对应的验证集
    
    #TextDataset为上面定义的一个继承的
    #这里面train_dataset.get_keys()的对应值为
    #valid_dataset.get_keys()的对应值为
    #[ 253  226  203  876  211  117  689  320  466  111  724  101  593  175
    #257  684  560  190  365  219  111  541  120  131   92  368  709  347
    # 349  433  292  817  333  149  299  547  174  714  266  206  305  723
    #579  650  817  300  241  305  613  162  236  240  765  495  452  151
    #103  983  285  424  187  201   95  183  256  262  171  622  148  216
    #265  310 1011  273  167  121  339  232  179  196  261  429  167  294
    #70  383  615  435  211  225  302  138   70  310   70  725  148  628
    #139  441  965  491  197  169  233   72  213  134  400  294  559  182
    #514  145  183  738  553  416  389  171  347  286  732  372   82  712
    #87  366  110  425  126  502 1024  386  224  308  567  253  608  405
    #582  578  365  345   68  310 1024  344  112  938  362  297  588  125
    #它这个操作是先从中抽取出句子，然后使用get_keys()方法获得问句和答句的加和矩阵
    
    #针对valid_dataset数据操作的时候valid_dataset = TextDataset(valid_fold_x)
    #此时TextDataset之中只有对应的一个数值，所以内容就是valid_fold_x的长度
    print('sampler = ')
    train_sampler = BucketSampler(train_dataset, train_dataset.get_keys(),
                                  bucket_size=batch_size * 20, batch_size=batch_size)
    #bucket_size = 128*20 = 2560,batch_size = 128
    valid_sampler = BucketSampler(valid_dataset, valid_dataset.get_keys(),
                                  batch_size=batch_size, shuffle_data=False)
    #batch_size = 128,
    print('data loader')
    #BucketSampler对应的初始化类型为
    #def __init__(self, data_source, sort_keys, bucket_size=None, batch_size=1048, shuffle_data=True):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False,
                              sampler=train_sampler, num_workers=0, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False,
                              sampler=valid_sampler, collate_fn=collate_fn)
    #这里DataLoader没有调用任何内部函数，只是将相应的数值取出来，
    print('embedding_matrix = ')
    print(embedding_matrix)
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    #embedding_matrix = (47462,300),这里的embedding_matrix为之前使用
    #fasttext.pkl+w2vec训练得到的最终结果
    model = LstmModel(embedding_matrix).to(device)
    #初始化在这里调用LstmModel(embedding_matrix)的初始化函数
    model.zero_grad()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    ema_model = copy.deepcopy(model)
    #copy浅拷贝，没有拷贝子对象，所以原始数据改变，子对象会改变
    #deepcopy深拷贝，包含对象里面的子对象的拷贝，所以原始对象的改变
    #不会造成深拷贝里面任何子元素的改变
    print('eval = ')
    ema_model.eval()
    
    ema_n = int(len(train_loader.dataset) / (updates_per_epoch * batch_size))
    ema = EMA(model, mu, n=ema_n)

    for epoch in range(n_epochs):
        epoch_start_time = time.time()
        model.train()
        for index, x_batch, y_batch in train_loader:
            x_batch = (x.to(device) for x in x_batch)
            y_batch = y_batch.to(device)
            emb_batch = train_fold_emb[list(index)].to(device)
            y_preds = model(*x_batch, emb_batch)
            
            loss = nn.BCEWithLogitsLoss()(y_preds, y_batch)
            loss.backward()
            optimizer.step()
            model.zero_grad()
            ema.on_batch_end(model)
        
        valid_preds = predict(model, valid_loader, valid_fold_emb, device=device)
        scores = get_scores(valid_fold_y, valid_preds)
        score = np.mean(list(scores.values()))
        fold_scores.append({
            'fold': i + 1,
            'epoch': epoch + 1,
            'score': score
        })
        elapsed_time = time.time() - epoch_start_time
        print('Epoch {}/{} \t score: {:.4f} \t time: {:.2f}s'.format(
            epoch + 1, n_epochs, score, elapsed_time))
        ema.on_epoch_end(model)
        
    ema.set_weights(ema_model)
    ema_model.flatten_parameters()
    
    train_preds[valid_idx] = valid_preds
    ema_valid_preds = predict(ema_model, valid_loader, valid_fold_emb, device=device)
    ema_train_preds[valid_idx] = ema_valid_preds

    cv_scores.append(score)
    ema_scores = get_scores(valid_fold_y, ema_valid_preds)
    ema_score = np.mean(list(ema_scores.values()))
    print(f'EMA score: {ema_score:.4f}')
    ema_cv_scores.append(ema_scores)
    
    test_preds += predict(model, test_loader, test_text_emb, device=device) / n_splits
    ema_test_preds += predict(ema_model, test_loader, test_text_emb, device=device) / n_splits

fold 1
train_fold_emb = 
torch.Size([4863, 1024])
valid_fold_emb = 
torch.Size([1216, 1024])
TextDataset __init__
TextDataset __init__
sampler = 
TextDataset get_keys
BucketSampler __init__
after get_keys
[282 387 216 ... 178 470 313]
sort_keys.shape = 
(4863,)
self.bucket_size = 
2560
TextDataset get_keys
BucketSampler __init__
after get_keys
[370 359 165 ... 245 351 382]
sort_keys.shape = 
(1216,)
self.bucket_size = 
1216
BucketSampler prepare_buckets
indices = 
[   0    1    2 ... 1213 1214 1215]
lens = 
[-370 -359 -165 ... -245 -351 -382]
self.bucket_size = 
1216
BucketSampler divide_chunks
chunk_index = 
[   0    1    2 ... 1213 1214 1215]
chunk = 
[-370 -359 -165 ... -245 -351 -382]
indices_sorted = 
[540 941 282 ... 565 327 493]
chunk 132 = 
-300
BucketSampler divide_chunks
u = 
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49 

embedding_matrix.shape1 = 
300
LstmUnit __init__
embedding_matrix.shape1 = 
300
eval = 
TextDataset __len__


NameError: name 'EMA' is not defined

In [ ]:
fig = px.line(pd.DataFrame(fold_scores), x='epoch', y='score', color='fold')
fig.show()

In [ ]:
ema_cv_scores = pd.DataFrame(ema_cv_scores).mean().reset_index()
ema_cv_scores.columns = ['target_name', 'score']
ema_cv_scores.sort_values('score', inplace=True)
fig = px.bar(ema_cv_scores, x='score', y='target_name', orientation='h')
fig.show()

In [ ]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[target_names] = ema_test_preds
submission.to_csv('submission.csv', index=False)

In [ ]:
cv_score = np.mean(cv_scores)
ema_score = ema_cv_scores['score'].mean()
print(f'CV score: {cv_score:.4f}')
print(f'EMA score: {ema_score:.4f}')

In [ ]:
class OptimizedRounder(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.threshold = [0., 1.]
        self.ab_start = [(0., 0.2), (0.8, 1.)]
    
    def fit(self, train_labels, train_preds):
        assert train_labels.shape == train_preds.shape
        assert train_labels.ndim == 1
        
        self.best_score = self.score(train_labels, train_preds)
        self._golden_section_search(train_labels, train_preds, 0)  # lower threshold
        score = self.score(train_labels, train_preds)
        if score > self.best_score + 1e-3:
            self.best_score = score
        else:
            self.threshold[0] = 0.
        
        self._golden_section_search(train_labels, train_preds, 1)  # higher threshold
        score = self.score(train_labels, train_preds)
        if score > self.best_score + 1e-3:
            self.best_score = score
        else:
            self.threshold[1] = 1.

    def _golden_section_search(self, train_labels, train_preds, idx):
        # idx == 0 -> lower threshold search
        # idx == 1 -> higher threshold search
        golden1 = 0.618
        golden2 = 1 - golden1
        for _ in range(10):
            a, b = self.ab_start[idx]
            # calc losses
            self.threshold[idx] = a
            la = -self.score(train_labels, train_preds)
            self.threshold[idx] = b
            lb = -self.score(train_labels, train_preds)
            for _ in range(20):
                # choose value
                if la > lb:
                    a = b - (b - a) * golden1
                    self.threshold[idx] = a
                    la = -self.score(train_labels, train_preds)
                else:
                    b = b - (b - a) * golden2
                    self.threshold[idx] = b
                    lb = -self.score(train_labels, train_preds)

    def transform(self, preds):
        transformed = np.clip(preds, *self.threshold)
        if np.unique(transformed).size == 1:
            return preds
        return transformed
        
    def score(self, labels, preds):
        p = self.transform(preds)
        score = scipy.stats.spearmanr(labels, p)[0]
        return score

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=1029)
train_scores = []
valid_scores = []
train_optimized_scores = defaultdict(list)
valid_optimized_scores = defaultdict(list)
thresholds = defaultdict(list)

for train_idx, valid_idx in kf.split(train_y):
    train_fold_preds, train_fold_y = ema_train_preds[train_idx], train_y[train_idx]
    valid_fold_preds, valid_fold_y = ema_train_preds[valid_idx], train_y[valid_idx]
    train_scores.append(get_scores(train_fold_y, train_fold_preds))
    valid_scores.append(get_scores(valid_fold_y, valid_fold_preds))
    
    for i, target_name in enumerate(target_names):
        optimizer = OptimizedRounder()
        optimizer.fit(train_y[train_idx, i], ema_train_preds[train_idx, i])
        train_score = optimizer.score(train_y[train_idx, i], ema_train_preds[train_idx, i])
        valid_score = optimizer.score(train_y[valid_idx, i], ema_train_preds[valid_idx, i])
        train_optimized_scores[target_name].append(train_score)
        valid_optimized_scores[target_name].append(valid_score)
        thresholds[target_name].append(optimizer.threshold)

In [ ]:
scores = pd.DataFrame(valid_optimized_scores).mean().loc[ema_cv_scores['target_name']].reset_index()
scores.columns = ['target_name', 'score']
fig = px.bar(scores, x='score', y='target_name', orientation='h')
fig.show()

In [ ]:
train_score = pd.DataFrame(train_scores).values.mean()
valid_score = pd.DataFrame(valid_scores).values.mean()
train_optimized_score = np.mean(list(train_optimized_scores.values()))
valid_optimized_score = np.mean(list(valid_optimized_scores.values()))
print(f'train score: {train_score:.4f} -> {train_optimized_score:.4f}')
print(f'valid score: {valid_score:.4f} -> {valid_optimized_score:.4f}')

In [ ]:
for i, target_name in enumerate(target_names):
    optimizer = OptimizedRounder()
    optimizer.threshold = np.mean(thresholds[target_name], axis=0)
    ema_test_preds[:, i] = optimizer.transform(ema_test_preds[:, i])

In [ ]:
.submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[target_names] = ema_test_preds
submission.to_csv('submission.csv', index=False)
print(f'all processes done in {(time.time() - start_time) / 60:.2f} min.')